In [1]:
import numpy as np
import pandas as pd
import pooch
from scipy.optimize import root

In [2]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [3]:
bc_1850 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'1850'].values[0]
oc_1850 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'1850'].values[0]
so2_1850 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'1850'].values[0]

bc_2014 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'2014'].values[0]
oc_2014 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'2014'].values[0]
so2_2014 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'2014'].values[0]

In [4]:
bc_1850

np.float64(2.5711244787427185)

In [5]:
bc_2014

np.float64(9.744379658087515)

In [6]:
# Fiona reports IRF as ERFari and RA as ERFaci, where IRF is calculated using the Ghan 2012 method (2013 I think).
# This is same as Thornhill et al. 2021.

# BUT! Zelinka et al. 2023 states that APRP is not the same as Ghan. So we are not comparing apples with apples
target_erfari_so2 = -0.49
target_erfari_bc = +0.38
target_erfari_oc = -0.14
target_erfari = -0.15
target_erfaci_so2 = -0.88
target_erfaci_bc = -0.01
target_erfaci_oc = -0.08
target_erfaci = -0.99
target_albedo = +0.05

# O'Connor et al. 2021: https://acp.copernicus.org/articles/21/1211/2021/acp-21-1211-2021.html
# Individual forcing experiments:
# Sulfate ERF = -1.37
# BC ERF      = +0.37
# OC ERF      = -0.22
# total       = -1.22

# All species varied together = -1.09

# Thornhill et al. 2021 https://acp.copernicus.org/articles/21/853/2021/acp-21-853-2021.html

# Fiona sweeps up ERFcs,af into the RA term, which is dominated by ERFaci. Ghan attributes this to surface albedo,
# Thornhill says yes albedo, plus other tropospheric adjustments.

In [7]:
-0.49 + 0.38 - 0.14

-0.25

In [8]:
-0.88 - 0.01 - 0.08

-0.97

In [9]:
def erf_rootfinder(x):
    alpha_so2, alpha_bc, alpha_oc, s_so2, s_bc, s_oc, beta = x
    erfari_so2 = (so2_2014 - so2_1850) * alpha_so2
    erfari_bc = (bc_2014 - bc_1850) * alpha_bc
    erfari_oc = (oc_2014 - oc_1850) * alpha_oc
    erfaci_so2 = beta * (np.log(1 + s_so2 * so2_2014) - np.log(1 + s_so2 * so2_1850))
    erfaci_bc = beta * (np.log(1 + s_bc * bc_2014) - np.log(1 + s_bc * bc_1850))
    erfaci_oc = beta * (np.log(1 + s_oc * oc_2014) - np.log(1 + s_oc * oc_1850))
    erfaci = beta * (np.log(1 + s_so2 * so2_2014 + s_bc * bc_2014 + s_oc * oc_2014) - np.log(1 + s_so2 * so2_1850 + s_bc * bc_1850 + s_oc * oc_1850))
    return np.array(
        [
            erfari_so2 - target_erfari_so2, 
            erfari_bc - target_erfari_bc, 
            erfari_oc - target_erfari_oc, 
            erfaci_so2 - target_erfaci_so2, 
            erfaci_bc - target_erfaci_bc, 
            erfaci_oc - target_erfaci_oc, 
            erfaci - target_erfaci
        ]
    )

In [10]:
solution = root(
    erf_rootfinder, 
    np.array(
        [
            -0.0043,
            0.05158,
            -0.0052,
            0.037031928,
            0.001,
            0.016231426,
            -0.735769463,
        ],
    ),
    method='lm',
    options={'maxiter': 500000}
)

/var/folders/9c/9qs75_fd7sz60mklfvdt47z80000gp/T/ipykernel_90987/2798208126.py:6: RuntimeWarning: invalid value encountered in log
  erfaci_so2 = beta * (np.log(1 + s_so2 * so2_2014) - np.log(1 + s_so2 * so2_1850))
/var/folders/9c/9qs75_fd7sz60mklfvdt47z80000gp/T/ipykernel_90987/2798208126.py:9: RuntimeWarning: invalid value encountered in log
  erfaci = beta * (np.log(1 + s_so2 * so2_2014 + s_bc * bc_2014 + s_oc * oc_2014) - np.log(1 + s_so2 * so2_1850 + s_bc * bc_1850 + s_oc * oc_1850))


In [11]:
solution

 message: Both actual and predicted relative reductions in the sum of squares
            are at most 0.000000
 success: True
  status: 1
     fun: [ 5.551e-17  0.000e+00 -2.776e-17 -5.000e-03 -5.000e-03
           -5.000e-03  5.001e-03]
       x: [-4.476e-03  5.297e-02 -7.813e-03  5.030e-08  1.301e-08
            2.951e-08 -1.607e+05]
   cov_x: [[ 8.346e-05  1.834e-19 ...  1.715e-13  8.947e-01]
           [ 1.834e-19  1.943e-02 ...  1.127e-13  5.878e-01]
           ...
           [ 1.715e-13  1.127e-13 ...  3.178e-03  1.658e+10]
           [ 8.947e-01  5.878e-01 ...  1.658e+10  8.651e+22]]
  method: lm
    nfev: 22312
    fjac: [[-2.485e+07 -1.951e+06 ...  0.000e+00  8.227e-06]
           [-1.951e+06 -3.379e+06 ...  0.000e+00  6.858e-07]
           ...
           [ 0.000e+00  0.000e+00 ... -7.173e+00  4.874e-23]
           [ 8.227e-06  6.858e-07 ...  4.874e-23 -3.400e-12]]
    ipvt: [3 5 4 0 2 1 6]
     qtf: [-6.048e-09  3.204e-07  9.326e-08  5.530e-17 -2.690e-17
            2.960e-19

In [12]:
solution.x

array([-4.47634941e-03,  5.29745549e-02, -7.81293747e-03,  5.02952839e-08,
        1.30087418e-08,  2.95093423e-08, -1.60747888e+05])